In [1]:
# download corpus
!wget https://raw.githubusercontent.com/InsaneLife/ChineseNLPCorpus/master/NER/renMinRiBao/renmin.txt

--2020-04-24 05:18:52--  https://raw.githubusercontent.com/InsaneLife/ChineseNLPCorpus/master/NER/renMinRiBao/renmin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10648033 (10M) [text/plain]
Saving to: ‘renmin.txt’

renmin.txt          100%[===================>]  10.15M  --.-KB/s    in 0.07s   

2020-04-24 05:18:54 (146 MB/s) - ‘renmin.txt’ saved [10648033/10648033]



In [2]:
# clone pos model project
!git clone https://github.com/grallage/transformers-chinese-pos.git

Cloning into 'transformers-chinese-pos'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [6]:
# preprocess corpus and get train.txt, dev.txt, test.txt and labels.txt
!python ./transformers-chinese-pos/renmin_preprocess.py ./renmin.txt 128

corpus_size = 24998, train_size = 19998, dev_size = 2500


In [4]:
!pip install transformers
!pip install seqeval
!pip install fastprogress

     |████████████████████████████████| 573kB 25.2MB/s 
     |████████████████████████████████| 1.0MB 51.8MB/s 
     |████████████████████████████████| 890kB 61.9MB/s 
     |████████████████████████████████| 3.7MB 54.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=ca623c9adbe54f13652eac5655f98dac19bea39375f61d064195637a188079fc
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=ea3ef290467c77d2d75609bb3a9fcc302943f860c418b6565fa741b5bf8830bc
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [5]:
!ls -alh *.txt

-rw-r--r-- 1 root root 11M Apr 24 05:18 renmin.txt


In [0]:
# train model
!python3 ./transformers-chinese-pos/run_tf_ner.py --data_dir ./ \
--model_type bert \
--labels ./labels.txt \
--model_name_or_path bert-base-chinese \
--output_dir germeval \
--max_seq_length  128 \
--num_train_epochs 1 \
--per_device_train_batch_size 16 \
--save_steps 750 \
--seed 1 \
--do_train \
--overwrite_output_dir

2020-04-24 05:20:30.116657: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
W0424 05:20:35.390244 140623179540352 run_tf_ner.py:409] n_device: 1, distributed training: False, 16-bits training: False
I0424 05:20:36.348312 140623179540352 filelock.py:274] Lock 140620223392624 acquired on /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305.lock
I0424 05:20:36.348660 140623179540352 file_utils.py:479] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpa29kee6a


In [0]:
# test
!python3 ./transformers-chinese-pos/terminal_predict.py \
--model_type bert \
--labels ./labels.txt \
--output_dir germeval \
--max_seq_length  128 \